<table class="table table-bordered">
    <tr>
        <th style="width:250px"><img src='https://www.np.edu.sg/PublishingImages/Pages/default/odp/ICT.jpg' style="width: 100%; height: 125px; "></th>
        <th style="text-align:center;"><h1>Data Science Capstone</h1><h2>Assignment 1 </h2><h3>Diploma in Data Science</h3></th>
    </tr>
</table>

In [ ]:
# import the required packages

import pandas as pd
import numpy as np

# Visual
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import nltk
from collections import Counter
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.svm import SVR
sns.set(style="white")

In [ ]:
# LAST UPDATED 6/5/24
mp1_231223 = pd.read_csv('data/MacPherson_Rack_01 -  23 December 2023.csv')
mp1_050424 = pd.read_csv('data/MacPherson_Rack_01 - 5 April 2024.csv')
mp1_060424 = pd.read_csv('data/MacPherson_Rack_01 - 6 April 2024.csv')
mp1_100424 = pd.read_csv('data/MacPherson_Rack_01 - 10 April 2024.csv')
mp1_140424 = pd.read_csv('data/MacPherson_Rack_01 - 14 April 2024.csv')
mp1_150424 = pd.read_csv('data/MacPherson_Rack_01 - 15 April 2024.csv')
mp1_160424 = pd.read_csv('data/MacPherson_Rack_01 - 16 April 2024.csv')

### 1. Import files 

### Macpherson - 23 Dec 2023

In [ ]:
mp1_231223.shape

In [ ]:
mp1_231223.info()

In [ ]:
#Converting column name for appending files together later 
mp1_231223 = mp1_231223.rename(columns={'Buzzer (NIL)': 'Buzzer'})
mp1_231223

### Macpherson - 05 April 2024

In [ ]:
mp1_050424.shape

In [ ]:
mp1_050424.info()

In [ ]:
mp1_050424

### Macpherson - 06 April 2024

In [ ]:
mp1_060424.shape

In [ ]:
mp1_060424.info()

In [ ]:
#first 7 rows have '?' for NPK values
mp1_060424

### Macpherson - 10 April 2024

In [ ]:
mp1_100424.shape

In [ ]:
mp1_100424.info()

In [ ]:
mp1_100424

### Macpherson - 14 April 2024

In [ ]:
mp1_140424.shape

In [ ]:
mp1_140424.info()

In [ ]:
mp1_140424

In [ ]:
#Dropping the first column 
mp1_140424.drop(0, inplace=True) # Row of units and measurements
mp1_140424.reset_index(drop=True, inplace=True)
mp1_140424

### Macpherson - 15 April 2024

In [ ]:
mp1_150424.shape

In [ ]:
mp1_150424.info()

In [ ]:
mp1_150424.drop(0, inplace=True) # Row of units and measurements
mp1_150424.reset_index(drop=True, inplace=True)
mp1_150424

### Macpherson - 16 April 2024

In [ ]:
mp1_160424.shape

In [ ]:
mp1_160424.info()

In [ ]:
mp1_160424.drop(0, inplace=True) # Row of units and measurements
mp1_160424.reset_index(drop=True, inplace=True)
mp1_160424

### 2. Exploration

In [ ]:
#appending files together
df = pd.concat([mp1_231223, mp1_050424, mp1_060424, mp1_100424, mp1_140424, mp1_150424, mp1_160424], axis=0)
df

In [ ]:
df.shape

In [ ]:
df.info()

### Exploring objects 

In [ ]:
#Exploring grams of compost harvest but its not very useful to analyse  as there are too many columns with null values
#Since compost notes only has 1 value, we will not explore it too

label_freq = df['Grams of Compost Harvested'].value_counts() / len(df)
fig = label_freq.sort_values(ascending=False).plot.bar()

fig.axhline(y=0.05, color='red')

fig.set_ylabel('Percentage of items in each category')
fig.set_xlabel('Variable: Grams of Compost Harvested')
fig.set_title('Rare Categories')
plt.show()

In [ ]:
#Exporing number of worms 
#If using the column, clean to remove the word in the bracket and convert the 14.0 to 14.
label_freq = df['Number of Worms (non-counted)'].value_counts() / len(df)
fig = label_freq.sort_values(ascending=False).plot.bar()

fig.axhline(y=0.05, color='red')

fig.set_ylabel('Percentage of items in each category')
fig.set_xlabel('Variable: Num. of worms')
fig.set_title('Rare Categories')
plt.show()

In [ ]:
#Exploring Buzzer
label_freq = df['Buzzer'].value_counts() / len(df)
fig = label_freq.sort_values(ascending=False).plot.bar()

fig.axhline(y=0.05, color='red')

fig.set_ylabel('Percentage of items in each category')
fig.set_xlabel('Variable: Buzzer')
fig.set_title('Rare Categories')
plt.show()

### Exploring float variables

In [ ]:
# drop rows with no npk values
df = df.dropna(subset=['Nitrogen01']) 

# Convert List of columns to float
columns_to_convert = ['Phosphorous01', 'Phosphorous02', 'Nitrogen01', 'Nitrogen02', 
                      'Potassium01', 'Potassium02', 'Temp01', 'Hum01', 'Heat01', 
                      'SoilM01', 'SoilM02']

# Convert columns to float
for col in columns_to_convert:
    # Convert non-positive values (such as "?" and negative numbers) with NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

df

In [ ]:
df.info()

In [ ]:
def diagnostic_plots(df, variable):
    
    plt.figure(figsize=(15,6))
    plt.subplot(1, 3, 1)
    plt.rcParams["patch.edgecolor"] = "none"
    sns.histplot(df[variable], bins=30, kde = True, stat ="density", kde_kws=dict(cut = 3),alpha = 0.30)
    plt.title('Histogram')

    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])

    plt.show()
    
    # Count outliers using IQR method
    Q1 = df[variable].quantile(0.25)
    Q3 = df[variable].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[variable] < lower_bound) | (df[variable] > upper_bound)]

    print(f'Number of outliers in {variable}: {len(outliers)}')
    
diagnostic_plots(df, 'Phosphorous01')

In [ ]:
diagnostic_plots(df, 'Phosphorous02')

In [ ]:
diagnostic_plots(df, 'Nitrogen01')

In [ ]:
diagnostic_plots(df, 'Nitrogen02')

In [ ]:
diagnostic_plots(df, 'Potassium01')

In [ ]:
diagnostic_plots(df, 'Potassium02')

In [ ]:
diagnostic_plots(df, 'Temp01')

In [ ]:
diagnostic_plots(df, 'Hum01')

In [ ]:
diagnostic_plots(df, 'Heat01')

In [ ]:
diagnostic_plots(df, 'SoilM01')

In [ ]:
diagnostic_plots(df, 'SoilM02')

### Relationship between variables

In [ ]:
#Understanding the correlation of variables by showing the correlation matrix on a heatmap 
df_num = df.select_dtypes(['float64']).copy()

# Compute the correlation matrix
correlation_matrix = df_num.corr()

# Set up the matplotlib figure
plt.figure(figsize=(16, 16))

# Create a heatmap using seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)

# Show the plot
plt.title('Correlation Heatmap of Macpherson Rack 01 Numerical Features')
plt.show()

In [ ]:
# Calculate summary statistics for each feature
summary_stats = {}
for column in df_num:
    stats = df.groupby('Buzzer')[column].agg(['mean', 'median', 'std'])
    summary_stats[column] = stats

# Display summary statistics for each feature
for column, stats in summary_stats.items():
    print(f"Summary statistics for '{column}':")
    print(stats)
    print()


In [ ]:
# Exploring relationship between variables
#Phosphorus
plt.figure(figsize=(8, 6))
plt.scatter(df['Phosphorous01'], df['Phosphorous02'])
plt.title('Scatter Plot of Phosphorous01 vs Phosphorous02')
plt.xlabel('Phosphorous01')
plt.ylabel('Phosphorous02')
plt.grid(True)
plt.show()

In [ ]:
#Nitrogen
plt.figure(figsize=(8, 6))
plt.scatter(df['Nitrogen01'], df['Nitrogen02'])
plt.title('Scatter Plot of Nitrogen01 vs Nitrogen02')
plt.xlabel('Nitrogen01')
plt.ylabel('Nitrogen02')
plt.grid(True)
plt.show()

In [ ]:
#Potassium
plt.figure(figsize=(8, 6))
plt.scatter(df['Potassium01'], df['Potassium02'])
plt.title('Scatter Plot of Potassium01 vs Potassium02')
plt.xlabel('Potassium01')
plt.ylabel('Potassium02')
plt.grid(True)
plt.show()

In [ ]:
#Temp01 vs Heat01
plt.figure(figsize=(8, 6))
plt.scatter(df['Temp01'], df['Heat01'])
plt.title('Scatter Plot of Temp01 vs Heat01')
plt.xlabel('Temp01')
plt.ylabel('Heat01')
plt.grid(True)
plt.show()

### 3. Cleaning 

In [ ]:
#Dropping irrelevant features 
df = df.drop(columns=['Number of Worms (non-counted)', 'Light Intensity', 'Grams of Compost Harvested',
                                'Buzzer', 'Compost Notes', 'pH Rod 1', 'pH Rod 2'])
df

In [ ]:
#Dropping rows with null npk values
df['Timestamp']

In [ ]:
# Convert 'Timestamp' column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d %b %Y, %I:%M %p')

df

In [ ]:
#Creating features with timestamp
df['Time'] = df['Timestamp'].dt.time
df['Day'] = df['Timestamp'].dt.day

df

In [ ]:
df.info()

In [ ]:
df

### Treating missing and inaccurate data

In [ ]:
plt.figure(figsize=(12,8))  # Set the figure size

test = df[df['Day'] <= 6]
# Plot phosphorus01 against time
plt.plot(test['Timestamp'], test['Phosphorous01'], color='blue', linestyle='-')

# Set labels and title
plt.xlabel('Time')
plt.ylabel('Phosphorus01')
plt.title('Phosphorus01 over Time')

# Rotate x-axis labels for better readability if needed
plt.xticks(rotation=45)

# Show grid
plt.grid(True)

# Show plot
plt.show()

In [ ]:
# # Define the time period of sensor malfunction
start_time = pd.to_datetime('13:33:00').time()
end_time = pd.to_datetime('13:38:00').time()

# Flag the data points where day is 6th or 14th and time is between 13:33:00 and 13:38:00
sensor_malfunction = ((df['Day'] == 6) | (df['Day'] == 14)) & (df['Time'].apply(lambda x: x >= start_time) & (df['Time'].apply(lambda x: x <= end_time)))

NPK_columns = ['Phosphorous01', 'Phosphorous02', 'Nitrogen01', 'Nitrogen02', 'Potassium01' ,'Potassium02']
for column in NPK_columns:
    # Convert negative values to NaN
    df.loc[df[column] <= 0, column] = np.nan
    df.loc[sensor_malfunction, column] = np.nan
    
df.describe()

In [ ]:
# test moving average function
def moving_average(df):
    data = df.copy()
    for column in columns_to_convert:
        window_size = 3
        moving_avg = data[column].rolling(window=window_size, min_periods=1).mean()

        data[column] = np.where(data[column].isnull(), moving_avg, data[column])

    plt.figure(figsize=(12,8))

    test = data[data['Day'] <= 6]

    plt.plot(test['Timestamp'], test['Phosphorous01'], color='blue', linestyle='-')

    # Set labels and title
    plt.xlabel('Time')
    plt.ylabel('Phosphorus01')
    plt.title('Phosphorus01 over Time')

    plt.xticks(rotation=45)

    plt.grid(True)

    plt.show()
    return data

avg = moving_average(df)
avg.describe()

In [ ]:
# test interpolation
def interpolation(df):
    data = df.copy()
    for column in columns_to_convert:
        data[column] = data[column].interpolate(method='linear')
    plt.figure(figsize=(12,8))

    test = data[data['Day'] <= 6]
    plt.plot(test['Timestamp'], test['Phosphorous01'], color='blue', linestyle='-')

    plt.xlabel('Time')
    plt.ylabel('Phosphorus01')
    plt.title('Phosphorus01 over Time')

    plt.xticks(rotation=45)
    
    plt.grid(True)

    plt.show()
    return data

interpolate = interpolation(df)
interpolate.describe()

In [ ]:
# test forward and backward fill 
def fill(df):
    data = df.copy()
    for column in columns_to_convert:
        data[column] = data[column].fillna(method='ffill').fillna(method='bfill')
    plt.figure(figsize=(12,8))

    test = data[data['Day'] <= 6]

    plt.plot(test['Timestamp'], test['Phosphorous01'], color='blue', linestyle='-')

    plt.xlabel('Time')
    plt.ylabel('Phosphorus01')
    plt.title('Phosphorus01 over Time')

    plt.xticks(rotation=45)

    plt.grid(True)

    plt.show()
    return data

filled = fill(df)
filled.describe()

In [ ]:
# Chose interpolation in the end
df = interpolation(df)

In [ ]:
df.info()

### 4. Getting average NPK and Moisture values

In [ ]:
df['Mean_Phosphorous'] = df[['Phosphorous01', 'Phosphorous02']].mean(axis=1)
df['Mean_Nitrogen'] = df[['Nitrogen01', 'Nitrogen02']].mean(axis=1)
df['Mean_Potassium'] = df[['Potassium01', 'Potassium02']].mean(axis=1)
df['Mean_Moisture'] = df[['SoilM01', 'SoilM02']].mean(axis=1)
df['Mean_Moisture'] = df['Mean_Moisture'].replace(0.5, 1)
df = df.drop(columns = NPK_columns)
df = df.drop(columns = ['SoilM01', 'SoilM02'])
df.describe()

### 5. Preparing for Modelling

In [ ]:
df = df.drop(columns=['Timestamp', "Time", "Day", "Heat01"])
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='raise')
df.describe()

In [ ]:
correlation_matrix = df.corr()
colormap = plt.cm.BrBG
plt.figure(figsize=(16, 16))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", linewidths=.5)
plt.title('Correlation Heatmap of Macpherson Rack 01 Numerical Features')
plt.show()

In [ ]:
nitro = df.copy()
nitro = nitro.drop(columns=['Mean_Phosphorous', 'Mean_Potassium'])
target_corr = nitro.corr()["Mean_Nitrogen"].sort_values()
corr_df = pd.DataFrame({"Mean_Nitrogen": target_corr})
plt.figure(figsize=(15,15))
plt.title('Correlation to Nitrogen', size=20)
ax = sns.heatmap(corr_df, cmap="coolwarm", annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
phos = df.copy()
phos = phos.drop(columns=['Mean_Nitrogen', 'Mean_Potassium'])
target_corr = phos.corr()["Mean_Phosphorous"].sort_values()
corr_df = pd.DataFrame({"Mean_Phosphorous": target_corr})
plt.figure(figsize=(15,15))
plt.title('Correlation to Phosphorous', size=20)
ax = sns.heatmap(corr_df, cmap="coolwarm", annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
pot = df.copy()
pot = pot.drop(columns=['Mean_Nitrogen', 'Mean_Phosphorous'])
target_corr = pot.corr()["Mean_Potassium"].sort_values()
corr_df = pd.DataFrame({"Mean_Potassium": target_corr})
plt.figure(figsize=(15,15))
plt.title('Correlation to Potassium', size=20)
ax = sns.heatmap(corr_df, cmap="coolwarm", annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

### 6. Scaling

In [ ]:
df_scaled = df.copy()
df_scaled = df_scaled.drop(columns=['Mean_Nitrogen', 'Mean_Potassium', 'Mean_Phosphorous'])
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_scaled)
df_scaled = pd.DataFrame(df_scaled, columns = ['Temperature', 'Humidity', 'Moisture Presence'])
df_scaled

### 7. Modelling

In [ ]:
y = df[['Mean_Nitrogen', 'Mean_Phosphorous', 'Mean_Potassium']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_scaled, y, test_size=0.2, random_state=42)

### Random Forest

#### Baseline

In [ ]:
models = {}
for nutrient in y_train.columns:
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Testing models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluating models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')

#### GridSearch #1

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

In [ ]:
# Best hyperparameters for each nutrient
best_params = {
    'Mean_Nitrogen': {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100},
    'Mean_Phosphorous': {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100},
    'Mean_Potassium': {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
}

# Instantiate separate SVR models for each nutrient with best parameters
models = {}
for nutrient in y_train.columns:
    model = RandomForestRegressor(**best_params[nutrient], random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Evaluate the models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluate the models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')

#### GridSearch #2

In [ ]:
param_grid = {
    'n_estimators': [100, 250, 300],
    'max_depth': [9,10,11],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

In [ ]:
# Best hyperparameters for each nutrient
best_params = {
    'Mean_Nitrogen': {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250},
    'Mean_Phosphorous': {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100},
    'Mean_Potassium': {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
}

# Instantiate separate SVR models for each nutrient with best parameters
models = {}
for nutrient in y_train.columns:
    model = RandomForestRegressor(**best_params[nutrient], random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Evaluate the models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluate the models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')

#### Gridsearch #3

In [ ]:
param_grid = {
    'n_estimators': [100, 150, 250, 300],
    'max_depth': [9,10,11],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

#### GridSearch #4

In [ ]:
param_grid = {
    'n_estimators': [100, 250, 275, 300],
    'max_depth': [9,10,11],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

In [ ]:
# Best hyperparameters for each nutrient
best_params = {
    'Mean_Nitrogen': {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 275},
    'Mean_Phosphorous': {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100},
    'Mean_Potassium': {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
}

# Instantiate separate SVR models for each nutrient with best parameters
models = {}
for nutrient in y_train.columns:
    model = RandomForestRegressor(**best_params[nutrient], random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Evaluate the models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluate the models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')

In [ ]:
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Combine predictions (e.g., simple averaging)
ensemble_pred = y_pred.mean(axis=1)

# Evaluate the ensemble model
mae = mean_absolute_error(y_test.mean(axis=1), ensemble_pred)
r2 = r2_score(y_test.mean(axis=1), ensemble_pred)
print(f'Mean Absolute Error for the ensemble model: {mae}')
print(f'R-squared for the ensemble model: {r2}')

### XGBoost

In [ ]:
models = {}
for nutrient in y_train.columns:
    model = XGBRegressor(random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Testing models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluating models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')


#### GridSearch #1

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'eta': [0.3, 0.5, 0.7, 1],
    'subsample': [0.3, 0.5, 0.7, 1]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

In [ ]:
# Best hyperparameters for each nutrient
best_params = {
    'Mean_Nitrogen': {'max_depth': 10, 'eta': 0.3, 'n_estimators': 100, 'subsample': 1},
    'Mean_Phosphorous': {'max_depth': 5, 'eta': 0.3, 'n_estimators': 100, 'subsample': 1},
    'Mean_Potassium': {'max_depth': 10, 'eta': 0.3, 'n_estimators': 100, 'subsample': 1}
}

# Instantiate separate XGBoost models for each nutrient with best parameters
models = {}
for nutrient in y_train.columns:
    model = XGBRegressor(**best_params[nutrient], random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Evaluate the models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluate the models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')


#### GridSearch #2

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5,7,10],
    'eta': [0.2,0.3,0.4],
    'subsample': [0.9, 1]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

In [ ]:
# Best hyperparameters for each nutrient
best_params = {
    'Mean_Nitrogen': {'max_depth': 7, 'eta': 0.2, 'n_estimators': 100, 'subsample': 1},
    'Mean_Phosphorous': {'max_depth': 7, 'eta': 0.2, 'n_estimators': 100, 'subsample': 1},
    'Mean_Potassium': {'max_depth': 10, 'eta': 0.3, 'n_estimators': 100, 'subsample': 1}
}

# Instantiate separate XGBoost models for each nutrient with best parameters
models = {}
for nutrient in y_train.columns:
    model = XGBRegressor(**best_params[nutrient], random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Evaluate the models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluate the models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')

In [ ]:
param_grid = {
    'n_estimators': [50,100,150],
    'max_depth': [6,8,10],
    'eta': [0.1, 0.2,0.3],
    'subsample': [1]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

In [ ]:
# Best hyperparameters for each nutrient
best_params = {
    'Mean_Nitrogen': {'max_depth': 7, 'eta': 0.1, 'n_estimators': 100, 'subsample': 1},
    'Mean_Phosphorous': {'max_depth': 7, 'eta': 0.1, 'n_estimators': 100, 'subsample': 1},
    'Mean_Potassium': {'max_depth': 8, 'eta': 0.1, 'n_estimators': 100, 'subsample': 1}
}

# Instantiate separate XGBoost models for each nutrient with best parameters
models = {}
for nutrient in y_train.columns:
    model = XGBRegressor(**best_params[nutrient], random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Evaluate the models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluate the models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')

In [ ]:
param_grid = {
    'n_estimators': [90, 100,110],
    'max_depth': [7,8,9],
    'eta': [0, 0.05, 0.1],
    'subsample': [1]
}

# Dictionary to store best models
best_models = {}

# Train separate Random Forest models for each nutrient with GridSearchCV
for nutrient in y_train.columns:
    model = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    grid_search.fit(X_train, y_train[nutrient])
    best_models[nutrient] = grid_search.best_estimator_
    print(f"Best parameters for {nutrient}: {grid_search.best_params_}")

In [ ]:
# Best hyperparameters for each nutrient
best_params = {
    'Mean_Nitrogen': {'max_depth': 7, 'eta': 0.1, 'n_estimators': 110, 'subsample': 1},
    'Mean_Phosphorous': {'max_depth': 7, 'eta': 0.1, 'n_estimators': 90, 'subsample': 1},
    'Mean_Potassium': {'max_depth': 8, 'eta': 0.1, 'n_estimators': 90, 'subsample': 1}
}

# Instantiate separate XGBoost models for each nutrient with best parameters
models = {}
for nutrient in y_train.columns:
    model = XGBRegressor(**best_params[nutrient], random_state=42)
    model.fit(X_train, y_train[nutrient])
    models[nutrient] = model

# Evaluate the models
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Evaluate the models
for nutrient, model in models.items():
    mae = mean_absolute_error(y_test[nutrient], y_pred[nutrient])
    r2 = r2_score(y_test[nutrient], y_pred[nutrient])
    print(f'Mean Absolute Error for {nutrient}: {mae}')
    print(f'R-squared for {nutrient}: {r2}\n')

In [ ]:
y_pred = pd.DataFrame()
for nutrient, model in models.items():
    y_pred[nutrient] = model.predict(X_test)

# Combine predictions (e.g., simple averaging)
ensemble_pred = y_pred.mean(axis=1)

# Evaluate the ensemble model
mae = mean_absolute_error(y_test.mean(axis=1), ensemble_pred)
r2 = r2_score(y_test.mean(axis=1), ensemble_pred)
print(f'Mean Absolute Error for the ensemble model: {mae}')
print(f'R-squared for the ensemble model: {r2}')

### Isolation forest

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
# Scale the NPK deviation features
scaler = MinMaxScaler()
desired_ratio = {'Nitrogen': 2, 'Phosphorus': 1, 'Potassium': 1}
# Scale NPK deviation features separately
scaled_mean_nitrogen = scaler.fit_transform(df['Mean_Nitrogen'].values.reshape(-1, 1)).flatten()
scaled_mean_phosphorus = scaler.fit_transform(df['Mean_Phosphorous'].values.reshape(-1, 1)).flatten()
scaled_mean_potassium = scaler.fit_transform(df['Mean_Potassium'].values.reshape(-1, 1)).flatten()

# Calculate NPK deviation using scaled mean values
nitrogen_deviation = abs(scaled_mean_nitrogen - desired_ratio['Nitrogen'])
phosphorus_deviation = abs(scaled_mean_phosphorus - desired_ratio['Phosphorus'])
potassium_deviation = abs(scaled_mean_potassium - desired_ratio['Potassium'])

# Combine scaled mean and deviation with environmental features
combined_df = pd.DataFrame({
    'Temp01': df['Temp01'],
    'Hum01': df['Hum01'],
    'Moisture': df['Mean_Moisture'],
    'NPK_Nitrogen_Deviation': nitrogen_deviation,
    'NPK_Phosphorus_Deviation': phosphorus_deviation,
    'NPK_Potassium_Deviation': potassium_deviation
})

# Initialize and train the Isolation Forest model with combined data
isolation_forest = IsolationForest(contamination=0.05, random_state=49)

isolation_forest.fit(combined_df)


# Predict outliers/anomalies (1 for inliers, -1 for outliers)
predictions = isolation_forest.predict(combined_df)

# Identify outliers/anomalies in the original DataFrame
outliers = df[predictions == -1]
inliers = df[predictions == 1]
anomaly_scores = isolation_forest.decision_function(combined_df)

In [ ]:
outliers.info()

In [ ]:
outliers.describe() #78 outliers

In [ ]:
inliers = df[predictions == 1]
inliers.describe()

In [ ]:
import shap

# Train your anomaly detection model (e.g., Isolation Forest) and generate anomaly scores

# Identify top outliers based on anomaly scores
sorted_indices = np.argsort(anomaly_scores)
sorted_indices = sorted_indices[::-1]
top_outlier_indices = sorted_indices[:1]
top_outliers = combined_df.iloc[top_outlier_indices]

# Calculate SHAP values for the selected outlier data points
explainer = shap.Explainer(isolation_forest)  # Replace 'model' with your anomaly detection model
shap_values = explainer.shap_values(top_outliers)  # Replace 'outlier_data' with your selected outlier data points

# Visualize SHAP explanations for individual outlier predictions
shap.summary_plot(shap_values, features=top_outliers)
